In [47]:
!pip install selenium-wire selenium webdriver-manager fake_useragent 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 363.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 498.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00a 0:00:01


In [48]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, TimeoutException, ElementNotInteractableException
from bs4 import BeautifulSoup
import time
import pandas as pd

In [49]:
from urllib.parse import urlparse, urljoin
url = 'https://www.tripadvisor.com/Restaurant_Review-g47958-d10624330-Reviews-MIX_Kitchen_and_Bar-Ithaca_Finger_Lakes_New_York.html'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(4)
driver.find_element(By.ID, "onetrust-accept-btn-handler").click()

In [50]:
from fake_useragent import UserAgent
import time
import random 

ua = UserAgent()
headers = {
    "User-Agent": ua.random,
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1"
    }
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/>
  <link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/>
  <link href="https://static.tacdn.com/img2/brand_refresh/application_icons/touch-icon.png" rel="apple-touch-icon-precomposed"/>
  <link href="https://static.tacdn.com/img2/brand_refresh/application_icons/touch-icon-180x180.png" rel="apple-touch-icon-precomposed" sizes="180x180"/>
  <meta content="#34e0a1" name="theme-color"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="TripAdvisor" property="al:ios:app_name"/>
  <meta content="284876795" property="al:ios:app_store_id"/>
  <meta content="284876795" name="twitter:app:id:ipad" property="twitter:app:id:ipad"/>
  <meta content="284876795" name="twitter:app:id:iphone" property="twitter:app:id:iphone"/>
  <meta content="tripadvisor://w

In [55]:
def get_driver():
    """Initialize and configure Chrome WebDriver"""
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--lang=en-US')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

In [56]:
def accept_cookies(driver):
    """Handle cookie acceptance popup"""
    try:
        cookie_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )
        cookie_btn.click()
        print("Cookies accepted")
    except Exception as e:
        print(f"Cookie acceptance failed: {str(e)}")

In [ ]:
def extract_review_info(review):
    try:
        name = review.find_element(By.XPATH, ".//a[contains(@href, '/Profile')]").text
    except NoSuchElementException:
        name = ''
    try:
        date_raw = review.find_element(By.CLASS_NAME, "aVuQn").text
        date = date_raw.split("\n")[0].strip()
    except NoSuchElementException:
        date = ''
    


In [65]:
def scrape_reviews(driver):
    driver = get_driver()
    accept_cookies(driver)
    driver.get(url)
    time.sleep(random.uniform(7, 14))

    all_reviews = []

    while True:
        time.sleep(random.uniform(2, 5))

        review_sections = driver.find_elements(By.XPATH, "//div[contains(@class, 'Gatzn')]")
        print(f"Found {len(review_sections)} reviews.")

        if not review_sections:
            print("No reviews found on this page.")
            break
        
        break
review_sections = driver.find_elements(By.XPATH, "//div[contains(@class, 'Gatzn')]")
print(f"Found {len(review_sections)} reviews.")


MaxRetryError: HTTPConnectionPool(host='localhost', port=52057): Max retries exceeded with url: /session/13b21ef435c8149c1dbd39f0ebc2e6c3/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12e118080>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [62]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time

def solve_tripadvisor_verification(driver):
    """
    Solves TripAdvisor's 'slide to complete' verification puzzle.
    Args:
        driver: Selenium WebDriver instance
    Returns:
        bool: True if verification was attempted, False if puzzle not found
    """
    try:
        # Wait for verification elements to load
        time.sleep(2)
        
        # Locate the slider track and handle
        slider_track = driver.find_element(
            By.XPATH,
            '//div[contains(@class, "slider-track") or contains(@class, "puzzle-container")]'
        )
        slider_handle = driver.find_element(
            By.XPATH,
            '//div[contains(@class, "slider-handle") or contains(@class, "puzzle-piece")]'
        )
        
        # Calculate sliding distance (entire width of the track)
        track_width = slider_track.size['width']
        
        # Perform the slide action
        actions = ActionChains(driver)
        actions.click_and_hold(slider_handle)\
               .move_by_offset(track_width, 0)\
               .release()\
               .perform()
        
        # Wait for verification to complete
        time.sleep(3)
        return True
        
    except NoSuchElementException:
        print("Verification puzzle not found")
        return False
    except Exception as e:
        print(f"Error solving verification: {e}")
        return False

In [63]:
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, urlunparse
import re 

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)
def get_total_reviews(driver):
    "Extracts the total amount of reviews"
    try:
        reviews_span = driver.find_element(
            By.XPATH, 
            '//span[@data-automation="reviewCount"]'
        )
        
        if not reviews_span.text:
            reviews_span = driver.find_element(
                By.XPATH,
                '//span[contains(@class, "biGQs") and contains(@class, "KxBGd")]'
            )
        
        return int(reviews_span.text.split()[0])
        
    except NoSuchElementException:
        print("Review count element not found")
        return None
    except Exception as e:
        print(f"Error extracting reviews: {e}")
        return None
total_reviews = get_total_reviews(driver)

total = get_total_reviews(driver)
if total:
    print(f"Total reviews: {total}")
    # Pagination logic would go here
else:
    print("Failed to get review count")

driver.quit()

KeyboardInterrupt: 

In [60]:
def click_next_page(url, reviews_per_page=15):
    """Parses the URL by switching pages"""
    parts = url.split("-")

    if "or" not in url:
        time.sleep(2)  # <--- Short delay to allow page update
        next_offset = reviews_per_page
        parts.insert(parts.index("Reviews") + 1, f"or{next_offset}")
    else: 
        for i, part in enumerate(parts):
            if part.startswith("or"):
                current_offset = int(part.replace("or", ""))
                next_offset = current_offset + reviews_per_page
                parts[i] = f"or{next_offset}"
                break
    
    if next_offset is not None and next_offset < total_reviews:
        if "or" not in current_url:
            parts.insert(parts.index("Reviews") + 1, f"or{next_offset}")
        else:
            parts[i] = f"or{next_offset}"
        return "-".join(parts)
    else:
        return None